# Amazon Nova Meta Prompter

Transform any prompt to align with Amazon Nova guidelines.

This tool analyzes your existing prompts and adapts them for Amazon Nova capabilities including:
- Up to 1M token context
- Structured output formatting
- Chain-of-thought reasoning
- Clear section-based organization

## New: Intent-Based Pipeline

The meta prompter now includes automatic intent classification:
1. **Intent Classification** - Detects what your prompt needs (image understanding, RAG, tool use, etc.)
2. **Targeted Guidance Loading** - Only loads relevant prompt guidance for detected intents
3. **Optimized Transformation** - Transforms your prompt with focused best practices

## Setup

Configure AWS credentials via environment variables (recommended) or AWS CLI configuration.

**Option 1: Environment Variables (Recommended)**
```bash
import os
os.environ["AWS_PROFILE"]="your-profile-name"
os.environ["AWS_REGION"]="your-aws-region"
```

**Option 2: AWS CLI Configuration**
```bash
aws configure
```

## Install the Metaprompter

In [ ]:
%pip install nova_metaprompter-0.1.0-py3-none-any.whl

## Verify Credentials

Test your AWS credentials and Bedrock access:

In [1]:
import boto3
from botocore.config import Config

try:
    # This will use AWS_PROFILE and AWS_REGION from environment
    client = boto3.client('bedrock-runtime', config=Config(
        connect_timeout=3600,  # 60 minutes
        read_timeout=3600,     # 60 minutes
        retries={'max_attempts': 1}
    ))
    print("✅ AWS credentials configured successfully")
    print(f"   Region: {client.meta.region_name}")
except Exception as e:
    print(f"❌ AWS credentials error: {e}")
    print("   Please set AWS_PROFILE and AWS_REGION environment variables")
    print("   Or configure using 'aws configure'")

✅ AWS credentials configured successfully
   Region: us-east-1


## Imports

Import the transform function and display utilities:

In [2]:
from IPython.display import display, HTML
from nova_metaprompter import transform_prompt

# Display helper functions
def display_section_header(title):
    display(HTML(f'<h3 style="color: #2E86AB; border-bottom: 2px solid #2E86AB; padding-bottom: 10px;">{title}</h3>'))

def display_original_prompt(prompt, title="Original Prompt"):
    display_section_header(title)
    display(HTML(f'<div style="background-color: #f8f9fa; padding: 15px; border-radius: 5px; border-left: 4px solid #6c757d;"><pre style="white-space: pre-wrap; font-family: monospace;">{prompt.strip()}</pre></div>'))

def display_intent_results(result):
    """Display intent classification and guidance loading results."""
    if 'intents' in result:
        intents_html = ', '.join([f'<span style="background-color: #e3f2fd; padding: 2px 8px; border-radius: 12px; margin: 2px;">{i}</span>' for i in result['intents']]) or '<em>None detected</em>'
        display(HTML(f'<h4 style="color: #1565C0;">🎯 Detected Intents</h4>'))
        display(HTML(f'<div style="background-color: #e3f2fd; padding: 15px; border-radius: 5px; margin-bottom: 10px;">{intents_html}</div>'))
    
    if 'api_capabilities' in result:
        enabled = [k for k, v in result['api_capabilities'].items() if v]
        if enabled:
            caps_html = ', '.join([f'<span style="background-color: #e8f5e9; padding: 2px 8px; border-radius: 12px; margin: 2px;">{c}</span>' for c in enabled])
            display(HTML(f'<h4 style="color: #2E7D32;">⚙️ API Capabilities Enabled</h4>'))
            display(HTML(f'<div style="background-color: #e8f5e9; padding: 15px; border-radius: 5px; margin-bottom: 10px;">{caps_html}</div>'))
    
    if 'intent_reasoning' in result and result['intent_reasoning']:
        display(HTML(f'<div style="background-color: #f5f5f5; padding: 10px; border-radius: 5px; margin-bottom: 10px; font-size: 0.9em;"><strong>Reasoning:</strong> {result["intent_reasoning"]}</div>'))
    
    if 'guidance_files' in result:
        files_html = ', '.join([f'<code>{f}</code>' for f in result['guidance_files']])
        display(HTML(f'<div style="background-color: #fff3e0; padding: 10px; border-radius: 5px; margin-bottom: 20px;"><strong>📚 Guidance Loaded:</strong> {files_html}</div>'))

def display_nova_migration_results(result):
    if "error" in result:
        display(HTML(f'<div style="color: red; padding: 10px; border: 1px solid red; border-radius: 5px;"><strong>Error:</strong> {result["error"]}</div>'))
        return
    
    # Show intent results if present
    if 'intents' in result:
        display_intent_results(result)
    
    if 'thinking' in result:
        display(HTML('<h4 style="color: #2E86AB;">🧠 Analysis & Thinking</h4>'))
        display(HTML(f'<div style="background-color: #f8f9fa; padding: 15px; border-radius: 5px; margin-bottom: 20px;">{result["steps"].replace(chr(10), "<br>")}</div>'))
    
    if 'nova_draft' in result:
        display(HTML('<h4 style="color: #A23B72;">📝 Nova-Aligned Draft</h4>'))
        display(HTML(f'<div style="background-color: #fff3e0; padding: 15px; border-radius: 5px; margin-bottom: 20px; border-left: 4px solid #A23B72;"><pre style="white-space: pre-wrap; font-family: monospace;">{result["nova_draft"]}</pre></div>'))
    
    if 'reflection' in result:
        display(HTML('<h4 style="color: #F18F01;">🤔 Reflection</h4>'))
        display(HTML(f'<div style="background-color: #fff8e1; padding: 15px; border-radius: 5px; margin-bottom: 20px;">{result["reflection"].replace(chr(10), "<br>")}</div>'))
    
    if 'nova_final' in result:
        display(HTML('<h4 style="color: #C73E1D;">✨ Final Nova-Aligned Prompt</h4>'))
        display(HTML(f'<div style="background-color: #e8f5e8; padding: 15px; border-radius: 5px; border-left: 4px solid #C73E1D;"><pre style="white-space: pre-wrap; font-family: monospace; font-weight: bold;">{result["nova_final"]}</pre></div>'))

def display_error(error_message, note=None):
    display(HTML(f'<div style="color: red; padding: 10px; border: 1px solid red; border-radius: 5px;"><strong>Error:</strong> {error_message}</div>'))
    if note:
        print(f"\nNote: {note}")

print("✅ Setup complete!")

✅ Setup complete!


---

# Transform a Prompt

The pipeline automatically:
1. Classifies your prompt's intents using Nova Lite
2. Loads only the relevant guidance files
3. Generates multiple candidate transformations and picks the best one

In [3]:
# Example: Image understanding with structured output
example_prompt = """
Analyze this image and return a JSON object with all detected objects, 
their locations, and confidence scores
"""

display_original_prompt(example_prompt, "Example: Image + Structured Output")
print("\n🔄 Running intent-based pipeline...")


🔄 Running intent-based pipeline...


In [ ]:
# Transform using the full pipeline
try:
    result = transform_prompt(
        "Act like you are a screen analyzer bot. Given a screenshot of a Windows "
        "desktop with a red circle marker, your task is to identify where the user "
        "clicked. The red circle indicates the location of a mouse click. Describe "
        "what UI element the user clicked on.",
    )
    display_nova_migration_results(result)
except Exception as e:
    display_error(str(e), "This requires AWS credentials and access to Bedrock service.")

---

# Try Your Own Prompt

Enter your prompt below and run the cell to transform it:

In [ ]:
# Your custom prompt here
your_prompt = """
Summarize the following Document: {document_text}
"""

display_original_prompt(your_prompt, "Your Prompt")
print("\n🔄 Running intent-based pipeline...")

In [ ]:
# Transform your prompt
try:
    your_result = transform_prompt(your_prompt.strip())
    display_nova_migration_results(your_result)
except Exception as e:
    display_error(str(e), "This requires AWS credentials and access to Bedrock service.")

---

# Batch Transform

Transform multiple prompts from a JSONL file. Each line should be a JSON object with a `"prompt"` field:

```json
{"id": "summarize", "prompt": "Summarize this document: {text}"}
{"id": "classify", "prompt": "Classify the sentiment of: {review}"}
```

**Candidates:** The `candidates` setting controls how many candidate transformations are generated per prompt. A judge then picks the best one. Set `candidates = 1` to skip candidate generation and judging (faster, cheaper). Higher values (e.g. 4) produce better results at the cost of more API calls.

In [ ]:
from pathlib import Path
from nova_metaprompter.batch_transform import load_prompts, save_results, transform_single

# -- Configure --
input_file = Path("prompts.jsonl")   # your input file
output_file = Path("results.jsonl")  # where to save results
candidates = 4                        # number of candidate transforms per prompt (1 = fastest)

# -- Load prompts --
prompts = load_prompts(input_file)
print(f"Loaded {len(prompts)} prompts")

# -- Transform each prompt --
results = []
for i, item in enumerate(prompts, 1):
    print(f"[{i}/{len(prompts)}] Transforming: {item['id']}...")
    result = transform_single(item, n=candidates)
    results.append(result)
    print(f"  Status: {result['status']}")

# -- Save results --
save_results(results, output_file)
print(f"\nSaved {len(results)} results to {output_file}")